In [2]:
import dask.dataframe as dd

In [3]:
def load_merged_data():
    """
    Load all merged data from S3.
    """
    merged_dtypes = {
    'ClaimID': 'object',
    'ClaimStartDt': 'object',
    'ClaimEndDt': 'object',
    'Provider' : 'object',
    'InscClaimAmtReimbursed' : 'float64',
    'AttendingPhysician' :'object',
    'OperatingPhysician' :'object',
    'OtherPhysician' :'object',
    'AdmissionDt'  :'object',
    'ClmAdmitDiagnosisCode' :'object',
    'DeductibleAmtPaid' :'float64',
    'IPAnnualReimbursementAmt': 'float64',
    'OPAnnualReimbursementAmt': 'float64',
    'DischargeDt' :'object',
    'ClmAdmitDiagnosisCode': 'object',
    'ClmDiagnosisCode_1': 'object',
    'ClmDiagnosisCode_2': 'object',
    'ClmDiagnosisCode_3': 'object',
    'ClmDiagnosisCode_4': 'object',
    'ClmDiagnosisCode_5': 'object',
    'ClmDiagnosisCode_6': 'object',
    'ClmDiagnosisCode_7': 'object',
    'ClmDiagnosisCode_8': 'object',
    'ClmDiagnosisCode_9': 'object',
    'ClmDiagnosisCode_10': 'object', 
    'DiagnosisGroupCode': 'object',
    'IPAnnualDeductibleAmt': 'float64',
    'OPAnnualDeductibleAmt': 'float64',
    }
    #date_columns_in = ['ClaimStartDt', 'ClaimEndDt', 'AdmissionDt', 'DischargeDt']
    clean_path = "s3://medicare-fraud-data-25-05-2025/clean/"
    df_train = dd.read_csv(clean_path+"train_full/*.csv", dtype=merged_dtypes)
    df_test = dd.read_csv(clean_path+"test_full/*.csv", dtype=merged_dtypes)
    print("Data loaded successfully")
    
    return (df_train, df_test)

In [4]:
import dask.dataframe as dd
df_train, df_test = load_merged_data()


Data loaded successfully


In [5]:

#def convert_dates(df):
#    """
#    Convert date columns to datetime format.
#    """
#    date_columns_in = ['ClaimStartDt', 'ClaimEndDt', 'AdmissionDt', 'DischargeDt']
#    for col in date_columns_in:
#        df[col] = dd.to_datetime(df[col], errors='coerce')
#    return df
    

In [6]:
#def revert_dates(df):
#    """
#    Konvertiert Datetime-Spalten zurück in Strings im ISO-Format (YYYY-MM-DD).
#    """
#    date_columns_in = ['ClaimStartDt', 'ClaimEndDt', 'AdmissionDt', 'DischargeDt']
#    for col in date_columns_in:
#        # Prüfen, ob Spalte im DataFrame existiert und vom Datetime-Typ ist
#        if col in df.columns:
#            df[col] = df[col].asstype('object')  # Konvertiert Datetime zurück in String
#    return df


In [7]:
#df_train=convert_dates(df_train)
#df_test=convert_dates(df_test)

In [8]:
physician_cols = [col for col in df_test.columns if "Physician" in col]
print(physician_cols)



['AttendingPhysician', 'OperatingPhysician', 'OtherPhysician']


In [9]:
# 1. Replacing NANs in all Physician Columns by Zero
cols_to_fill = ['AttendingPhysician', 'OperatingPhysician', 'OtherPhysician']
df_test[cols_to_fill] = df_test[cols_to_fill].fillna(0)
df_train[cols_to_fill] = df_train[cols_to_fill].fillna(0)


In [10]:
# 2. Sum of the Beneficiary Age for every Provider
bene_age_sum_per_prv = df_test.groupby("Provider")["Bene_Age"].sum().reset_index()
bene_age_sum_per_prv = bene_age_sum_per_prv.rename(columns={"Bene_Age": "Bene_Age_Sum"})


In [11]:
# 3. Number of Total Claims per Provider. The original Idea was to identify the Total Number of false Claims by a Provider. For that he subtract the number of fradulent claims from the number of total claims
total_claims_per_prv = df_test.groupby("Provider")["ClaimID"].count().reset_index()
total_claims_per_prv.columns = ["Provider", "TotalClaims"]
total_claims_per_prv

,Provider,TotalClaims
npartitions=1,,
,string,int64
,...,...


In [16]:
def total_claims_per_provider_physicians(df):
    """
    Compute total claims per provider for each physician type, and return one merged Dask DataFrame.
    """

    # Count total claims per provider-physician type
    att = df.groupby(["Provider", "AttendingPhysician"])["ClaimID"].count().reset_index()
    att = att.rename(columns={"ClaimID": "AttendingPhysician_TotalClaims"})

    #op = df.groupby(["Provider", "OperatingPhysician"])["ClaimID"].count().reset_index()
    #op = op.rename(columns={"ClaimID": "OperatingPhysician_TotalClaims"})
#
    #ot = df.groupby(["Provider", "OtherPhysician"])["ClaimID"].count().reset_index()
    #ot = ot.rename(columns={"ClaimID": "OtherPhysician_TotalClaims"})
#
    ## Now reduce these to provider-level totals by summing claims per provider
    att_sum = att.groupby("Provider")["AttendingPhysician_TotalClaims"].sum().reset_index()
    #op_sum = op.groupby("Provider")["OperatingPhysician_TotalClaims"].sum().reset_index()
    #ot_sum = ot.groupby("Provider")["OtherPhysician_TotalClaims"].sum().reset_index()

    # Merge safely
    #merged = att_sum.merge(op_sum, on="Provider", how="outer")
    #merged = merged.merge(ot_sum, on="Provider", how="outer")

    return att_sum



In [17]:
merged=total_claims_per_provider_physicians(df_test)

In [18]:
merged.head(5)

,Provider,AttendingPhysician_TotalClaims
0,PRV51002,205
1,PRV51006,102
2,PRV51009,39
3,PRV51010,38
4,PRV51018,190


In [19]:
# 7. Prv_Physician_Count
def count_unique_physicians(df, physician_col):
    """
    Count unique physicians for each provider.
    If multiple columns are provided, all unique physician IDs across them are counted.
    Works with Dask DataFrames.
    """
    if isinstance(physician_col, list):
        # Combine provider with all physician columns, then reshape and deduplicate
        dfs = []
        for col in physician_col:
            temp = df[["Provider", col]].rename(columns={col: "Physician"}).dropna()
            dfs.append(temp)
        
        combined = dd.concat(dfs)
        unique_counts = (
            combined.dropna()
            .drop_duplicates()
            .groupby("Provider")["Physician"]
            .nunique()
            .reset_index()
        )
        unique_counts = unique_counts.rename(columns={"Physician": "Prv_Physician_Count"})

    else:
        unique_counts = (
            df.groupby("Provider")[physician_col]
            .nunique()
            .reset_index()
            .rename(columns={physician_col: f"{physician_col}_Count"})
        )

    return unique_counts


In [22]:
pr_Attphysician_count = count_unique_physicians(df_test, "AttendingPhysician")
pr_OPphysician_count = count_unique_physicians(df_test, "OperatingPhysician")
pr_Otphysician_count = count_unique_physicians(df_test, "OtherPhysician")
pr_allphysician_count = count_unique_physicians(df_test, ["AttendingPhysician", "OperatingPhysician", "OtherPhysician"])


In [23]:
pr_allphysician_count

,Provider,Prv_Physician_Count
npartitions=3,,
,string,int64
,...,...
,...,...
,...,...


In [24]:
# 10. Provider_Insurance_Clam_Reimbursement_Amt
def calculate_provider_insurance_reimbursement(df):
    """
    Calculate the total insurance reimbursement amount per provider.
    """
    return df.groupby("Provider")["InscClaimAmtReimbursed"].sum().reset_index().rename(
        columns={"InscClaimAmtReimbursed": "Provider_Insurance_Claim_Reimbursement_Amt"}
    )
provider_insurance_reimbursement = calculate_provider_insurance_reimbursement(df_test)
provider_insurance_reimbursement

,Provider,Provider_Insurance_Claim_Reimbursement_Amt
npartitions=1,,
,string,float64
,...,...


In [25]:
# 11. Provider_Total_Patients
def calculate_provider_total_patients(df):
    """
    Calculate the total number of unique patients per provider.
    """
    return df.groupby("Provider")["BeneID"].nunique().reset_index().rename(
        columns={"BeneID": "Provider_Total_Patients"}
    )
provider_total_patients = calculate_provider_total_patients(df_test)
provider_total_patients

,Provider,Provider_Total_Patients
npartitions=1,,
,string,int64
,...,...


In [26]:
# 12. Provider_Total_Chronic_Alzheimer Patients

def calculate_provider_total_chronic_patients(df, chronic_cols):
    """
    Calculates the total number of patients per provider for each chronic condition.

    Parameters:
        df (Dask or Pandas DataFrame): Input beneficiary DataFrame
        chronic_cols (list of str): List of chronic condition columns (values should be 0 or 1)

    Returns:
        DataFrame with one row per provider and total counts of each chronic condition.
    """
    # Check if all columns exist
    missing = [col for col in chronic_cols if col not in df.columns]
    if missing:
        raise ValueError(f"The following columns are missing: {missing}")
    
    # Group and sum per provider
    agg_df = df.groupby("Provider")[chronic_cols].sum().reset_index()

    # Rename columns
    agg_df = agg_df.rename(columns={col: f"Provider_Total_{col}_Patients" for col in chronic_cols})

    return agg_df
chronic_cols = [
    "ChronicCond_Alzheimer",
    "ChronicCond_Heartfailure",
    "ChronicCond_KidneyDisease",
    "ChronicCond_Cancer",
    "ChronicCond_ObstrPulmonary",
    "ChronicCond_Depression",
    "ChronicCond_Diabetes",
    "ChronicCond_IschemicHeart",
    "ChronicCond_Osteoporasis",
    "ChronicCond_rheumatoidarthritis",
    "ChronicCond_stroke"
]

provider_total_chronic_patients = calculate_provider_total_chronic_patients(df_test, chronic_cols)
provider_total_chronic_patients


,Provider,Provider_Total_ChronicCond_Alzheimer_Patients,Provider_Total_ChronicCond_Heartfailure_Patients,Provider_Total_ChronicCond_KidneyDisease_Patients,Provider_Total_ChronicCond_Cancer_Patients,Provider_Total_ChronicCond_ObstrPulmonary_Patients,Provider_Total_ChronicCond_Depression_Patients,Provider_Total_ChronicCond_Diabetes_Patients,Provider_Total_ChronicCond_IschemicHeart_Patients,Provider_Total_ChronicCond_Osteoporasis_Patients,Provider_Total_ChronicCond_rheumatoidarthritis_Patients,Provider_Total_ChronicCond_stroke_Patients
npartitions=1,,,,,,,,,,,,
,string,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64
,...,...,...,...,...,...,...,...,...,...,...,...


In [27]:
# 14. count of diagnosis for every Provider
import dask.dataframe as dd

def count_diagnosis_per_provider(df, diagnosis_cols):
    """
    Count non-null occurrences of each diagnosis code per provider.
    
    Parameters:
        df (Dask DataFrame): Input DataFrame containing diagnosis codes
        diagnosis_cols (list of str): List of diagnosis code columns
    
    Returns:
        Dask DataFrame with counts of each diagnosis column per provider
    """
    # Start with the first column's counts
    result = df.groupby("Provider")[diagnosis_cols[0]].count().reset_index().rename(
        columns={diagnosis_cols[0]: f"{diagnosis_cols[0]}_Count"}
    )
    
    # Iterate through remaining diagnosis columns and join counts
    for col in diagnosis_cols[1:]:
        temp = df.groupby("Provider")[col].count().reset_index().rename(
            columns={col: f"{col}_Count"}
        )
        result = result.merge(temp, on="Provider", how="outer")

    return result
diagnosis_cols = [
    "ClmAdmitDiagnosisCode",
    "ClmDiagnosisCode_1",
    "ClmDiagnosisCode_2",
    "ClmDiagnosisCode_3"
]
diagnosis_counts_per_provider = count_diagnosis_per_provider(df_test, diagnosis_cols)
diagnosis_counts_per_provider



,Provider,ClmAdmitDiagnosisCode_Count,ClmDiagnosisCode_1_Count,ClmDiagnosisCode_2_Count,ClmDiagnosisCode_3_Count
npartitions=1,,,,,
,string,int64,int64,int64,int64
,...,...,...,...,...


In [ ]:
# 16. Real Average Claims per Provider we have to run this after merging the dataframes
#df_train['real_avg_claims_per_provider'] = df_train['TotalClaims'] - df_train['TotalClaims'].mean()

KeyError: 'TotalClaims'

In [33]:
# 16. Average of Claimcost for every Provider
df_train['real_avg_claim_cost_per_provider'] = df_train['Provider_Insurance_Claim_Reimbursement_Amt'] - df_train['Provider_Insurance_Claim_Reimbursement_Amt'].mean()  

KeyError: 'Provider_Insurance_Claim_Reimbursement_Amt'

In [ ]:
# 17. Median of the Claimscost for every Provider
df_train['real_median_claim_cost_per_provider'] = df_train['Provider_Insurance_Claim_Reimbursement_Amt'] - df_train['Provider_Insurance_Claim_Reimbursement_Amt'].median()

In [28]:
# 18. Most frequent Claimcodes for every Provider
from functools import reduce
import dask.dataframe as dd

def most_frequent_claim_codes(df, claim_code_cols):
    """
    Find the most frequent claim code for each provider across multiple columns.
    
    Parameters:
        df (Dask DataFrame): Input DataFrame containing claim codes
        claim_code_cols (list of str): List of claim code column names
    
    Returns:
        Dask DataFrame: Each row contains Provider and the most frequent code per claim column
    """
    results = []

    for col in claim_code_cols:
        # Count frequencies per Provider per code
        code_counts = (
            df.groupby(["Provider", col])
            .size()
            .reset_index()
            .rename(columns={0: "Count"})
        )

        # Sort within each partition, then drop duplicates to get most frequent
        most_frequent = (
            code_counts.map_partitions(lambda pdf: pdf.sort_values("Count", ascending=False))
            .drop_duplicates(subset="Provider")
            .rename(columns={col: f"{col}_Most_Frequent"})
            .drop(columns=["Count"])
        )

        results.append(most_frequent)

    # Merge all the most frequent codes per column
    final_result = reduce(lambda left, right: left.merge(right, on="Provider", how="outer"), results)

    return final_result

claim_code_cols = [
    "ClmAdmitDiagnosisCode",
    "ClmDiagnosisCode_1",
    "ClmDiagnosisCode_2",
    "ClmDiagnosisCode_3",
    
]
most_frequent_codes = most_frequent_claim_codes(df_test, claim_code_cols)
most_frequent_codes

,Provider,ClmAdmitDiagnosisCode_Most_Frequent,ClmDiagnosisCode_1_Most_Frequent,ClmDiagnosisCode_2_Most_Frequent,ClmDiagnosisCode_3_Most_Frequent
npartitions=1,,,,,
,string,string,string,string,string
,...,...,...,...,...


In [29]:
from functools import reduce

def most_frequent_physicians(df, physician_cols):
    """
    Find the most frequent physician for each provider across multiple physician columns.
    
    Parameters:
        df (Dask DataFrame): Input DataFrame containing provider and physician columns
        physician_cols (list of str): List of physician column names
    
    Returns:
        Dask DataFrame: Each row contains Provider and the most frequent physician per column
    """
    results = []

    for col in physician_cols:
        # Count frequencies per Provider per Physician
        physician_counts = (
            df.groupby(["Provider", col])
            .size()
            .reset_index()
            .rename(columns={0: "Count"})
        )

        # Sort by frequency, then get most frequent physician per provider
        most_frequent = (
            physician_counts.map_partitions(lambda pdf: pdf.sort_values("Count", ascending=False))
            .drop_duplicates(subset="Provider")
            .rename(columns={col: f"{col}_Most_Frequent"})
            .drop(columns=["Count"])
        )

        results.append(most_frequent)

    # Merge all the most frequent physician columns on Provider
    final_df = reduce(lambda left, right: left.merge(right, on="Provider", how="outer"), results)

    return final_df
physician_cols = [
    "AttendingPhysician",
    "OperatingPhysician",
    "OtherPhysician"
]
most_frequent_physicians_df = most_frequent_physicians(df_test, physician_cols)
most_frequent_physicians_df

,Provider,AttendingPhysician_Most_Frequent,OperatingPhysician_Most_Frequent,OtherPhysician_Most_Frequent
npartitions=1,,,,
,string,string,string,string
,...,...,...,...


In [40]:
# 16. Real Average Claims per Provider
def calculate_bene_amount(df):
    """
    Return a Dask DataFrame with BeneID, AllocatedAmount (as-is), and summed Deductible & Reimbursed amounts.

    Parameters:
        df (Dask DataFrame): Input with reimbursement and deductible fields

    Returns:
        Dask DataFrame with columns: BeneID, AllocatedAmount, DeductibleAmtPaid (sum), InscClaimAmtReimbursed (sum)
    """
    

    # Calculate AllocatedAmount (not to be summed)
    df["AllocatedAmount"] = df["IPAnnualReimbursementAmt"] + df["OPAnnualReimbursementAmt"]

    # Get first AllocatedAmount per BeneID (assuming same for all rows of that BeneID)
    allocated = df[["BeneID", "AllocatedAmount"]].drop_duplicates(subset="BeneID")

    # Sum the other columns per BeneID
    summed = df.groupby("BeneID")[["DeductibleAmtPaid", "InscClaimAmtReimbursed"]].sum().reset_index()

    # Merge
    result = allocated.merge(summed, on="BeneID", how="left")

    return result

bene_amount_df = calculate_remaining_amount(df_test)

In [41]:
bene_amount_df.head(5)

,BeneID,AllocatedAmount,DeductibleAmtPaid,InscClaimAmtReimbursed
0,BENE100001,2530.0,0.0,1100.0
1,BENE100002,14010.0,0.0,60.0
2,BENE100004,16150.0,1268.0,12500.0
3,BENE100010,14450.0,80.0,830.0
4,BENE100012,5870.0,1068.0,5070.0


In [44]:
remaining_avg_prv_df= dd.merge(
    df_test[["BeneID", "Provider"]],
    bene_amount_df,
    on="BeneID",
    how="left"
).groupby("Provider")["AllocatedAmount", 'DeductibleAmtPaid', "InscClaimAmtReimbursed"].mean().reset_index()
remaining_avg_prv_df = remaining_avg_prv_df.rename(
    columns={"AllocatedAmount": "Avg_alocated_Amount_Per_Provider", 'DeductibleAmtPaid': "Avg_Deductible_Amt_Paid_Per_Provider", "InscClaimAmtReimbursed": "Avg_InscClaimAmtReimbursed_Per_Provider" }
)
remaining_avg_prv_df.head(5)
remaining_avg_prv_df['per_utils_remaining_amount'] = (remaining_avg_prv_df["Avg_InscClaimAmtReimbursed_Per_Provider"] - remaining_avg_prv_df["Avg_Deductible_Amt_Paid_Per_Provider"]) /(remaining_avg_prv_df["Avg_alocated_Amount_Per_Provider"] - remaining_avg_prv_df["Avg_Deductible_Amt_Paid_Per_Provider"])

KeyError: 'BeneID'

In [38]:
remaining_avg_prv_df.head(5)

,Provider,Avg_alocated_Amount_Per_Provider,Avg_Deductible_Amt_Paid_Per_Provider,Avg_InscClaimAmtReimbursed_Per_Provider,per_utils_remaining_amount
0,PRV51002,7526.000000,72.409756,1322.585366,0.167728
1,PRV51006,6169.215686,75.156863,1617.254902,0.253049
2,PRV51009,5445.128205,121.384615,1767.435897,0.309191
3,PRV51010,7044.210526,449.684211,4747.105263,0.651665
4,PRV51018,7551.894737,53.968421,1332.000000,0.170451
